In [1]:
from __future__ import absolute_import, division, print_function
import pandas as pd

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

1.11.0


In [2]:
data = pd.read_csv(r'..\data\lastfm-360k-splitted\train.csv', sep='\t', header=None)
data = data.drop(data.columns[1], axis=1)
data.columns = ['user', 'artist', 'plays']

# Drop NaN columns
data = data.dropna()

data.head()

,user,artist,plays
0,aa3bf05c363a1f7b88750900c6e565657169619f,erykah badu,9
1,c6de0790ed225c7a9455051180437014b304a5d0,marlango,13
2,56438f603b828d8f782d1107dd5e4894ac4a06d6,a perfect circle,1026
3,007bc1593dc966671de398ca5fbca8d5b204e3ac,architecture in helsinki,98
4,a912750ce9d3001449aa5683a4f0bd123afb555f,dispatch,41


In [3]:
def construct_model(users, items, params):
  # type: (tf.Tensor, tf.Tensor, dict) -> tf.Tensor
  """Initialize NeuMF model.

  Args:
    users: Tensor of user ids.
    items: Tensor of item ids.
    params: Dict of hyperparameters.

  Raises:
    ValueError: if the first model layer is not even.
  """

  num_users = params["num_users"]
  num_items = params["num_items"]
  mf_regularization = params["mf_regularization"]
  mf_dim = params["mf_dim"]

  # Input variables
  user_input = tf.keras.layers.Input(tensor=users)
  item_input = tf.keras.layers.Input(tensor=items)

  # Initializer for embedding layers
  embedding_initializer = "glorot_uniform"

  # Embedding layers of GMF and MLP
  mf_embedding_user = tf.keras.layers.Embedding(
      num_users,
      mf_dim,
      embeddings_initializer=embedding_initializer,
      embeddings_regularizer=tf.keras.regularizers.l2(mf_regularization),
      input_length=1)
  mf_embedding_item = tf.keras.layers.Embedding(
      num_items,
      mf_dim,
      embeddings_initializer=embedding_initializer,
      embeddings_regularizer=tf.keras.regularizers.l2(mf_regularization),
      input_length=1)

  # GMF part
  mf_user_latent = mf_embedding_user(user_input)
  mf_item_latent = mf_embedding_item(item_input)
  # Element-wise multiply
  mf_vector = tf.keras.layers.multiply([mf_user_latent, mf_item_latent])

  # Print model topology.
  model = tf.keras.models.Model([user_input, item_input], mf_vector)
  model.summary()

  return model

In [10]:
users = tf.constant([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
items = tf.constant([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
params = {
    'num_users': 10,
    'num_items': 10,
    'mf_dim': 20,
    'mf_regularization': 0.1
}
model = construct_model(users, items, params)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (10,)                0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (10,)                0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (10, 20)             200         input_7[0][0]                    
__________________________________________________________________________________________________
embedding_7 (Embedding)         (10, 20)             200         input_8[0][0]                    
__________________________________________________________________________________________________
multiply_3

In [12]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
model.fit(steps_per_epoch=1)

IndexError: list index out of range